### Import modules

In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
from dataset import *

2022-09-30 20:13:41.722706: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-30 20:13:42.506018: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-30 20:13:42.506485: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-09-30 20:13:42.638891: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-30 20:13:46.446303: W tensorflow/stream_executor/platform/de

### Create dataset from test audio file


In [2]:
xf, X = create_dataset('audio/training_audio.wav', 2, 0.5)
print(X.shape)

(116, 1024, 169)


### Define some 'blocks' of NN code

In [73]:
def residual_block(inputs, input_shape, channels, block_name='ResBlock'):
    
    conv2d_1 = keras.layers.Conv2D(channels, kernel_size=(10, 1), padding='same', input_shape=input_shape)(inputs)

    batch_norm_1 = keras.layers.BatchNormalization()(conv2d_1)
    activation_1 = keras.layers.Activation('relu')(batch_norm_1)

    conv2d_2 = keras.layers.Conv2D(channels, kernel_size=10, padding='same')(activation_1)
    batch_norm_2 = keras.layers.BatchNormalization()(conv2d_2)

    add = keras.layers.Add()([batch_norm_2, inputs])
    activation_2 = keras.layers.Activation('relu')(add)
    return activation_2

def downscale_block(inputs, num_filters, block_name='DownscaleBlock'):

    downscale = keras.layers.Conv2D(num_filters, kernel_size=(10, 1), padding='same')(inputs)
    batch_norm = keras.layers.BatchNormalization()(downscale)
    activation = keras.layers.Activation('relu')(batch_norm)
    
    return activation

def upscale_block(inputs, target_size, num_filters, block_name='UpscaleBlock'):
    
    _, height, timesteps, channels = inputs.shape
    upscale = keras.layers.Conv2DTranspose(num_filters, (target_size - height + 1, 1))(inputs)
    batch_norm = keras.layers.BatchNormalization()(upscale)
    activation = keras.layers.Activation('relu')(batch_norm)

    return activation

### Define and train model

In [108]:
def encoder_lstm_decoder(spec_shape, start_filters):
    input_length =  spec_shape[0]
    time_steps = spec_shape[1]
    input_shape = (input_length, time_steps, 1)

    # Encoder
    inputs = keras.layers.Input(input_shape)                                                                    # input_length is power of two

    res_block_1 = residual_block(inputs, input_shape, 1, 'ResBlock1')
    max_pool_1 = keras.layers.MaxPool2D(pool_size=(2,1))(res_block_1)                 # (input_length/2,)
    down_scale_1 = downscale_block(max_pool_1, start_filters, 'DownscaleBlock1')

    res_block_2 = residual_block(down_scale_1, (input_length//2, time_steps, start_filters), start_filters, 'ResBlock2')
    max_pool_2 = keras.layers.MaxPool2D(pool_size=(2,1))(res_block_2)                 # (input_length/2,)
    down_scale_2 = downscale_block(max_pool_2, start_filters * 2, 'DownscaleBlock2')

    res_block_3 = residual_block(down_scale_2, (input_length//4, time_steps, start_filters*2), start_filters * 2, 'ResBlock3')
    max_pool_3 = keras.layers.MaxPool2D(pool_size=(2,1))(res_block_2)                 # (input_length/2,)
    down_scale_3 = downscale_block(max_pool_3, start_filters * 4, 'DownscaleBlock3')

    res_block_4 = residual_block(down_scale_3, (input_length//8, time_steps, start_filters*4), start_filters * 4, 'ResBlock4')
    max_pool_4 = keras.layers.MaxPool2D(pool_size=(2,1))(res_block_4)                 # (input_length/2,)
    down_scale_4 = downscale_block(max_pool_4, start_filters * 8, 'DownscaleBlock4')

    # Decoder
    _, height, timesteps, channels = down_scale_4.shape
    down_scale_4_flat = tf.reshape(down_scale_4, (-1, height * channels, timesteps))
    lstm = keras.layers.LSTM(input_length//16, return_sequences=True)(down_scale_4_flat)

    lstm_unflatten = tf.reshape(lstm, (-1, height, timesteps, channels))

    upscale_1 = upscale_block(lstm_unflatten, input_length//8, start_filters * 4, 'Upscale1')
    upscale_2 = upscale_block(upscale_1, input_length//4, start_filters * 2, 'Upscale2')
    upscale_3 = upscale_block(upscale_2, input_length//2, start_filters * 1, 'Upscale3')
    upscale_4 = upscale_block(upscale_3, input_length, start_filters, 'Upscale4')

    _, height, timesteps, channels = upscale_4.shape
    outputs = tf.reshape(upscale_4, (-1, height, timesteps))
    
    # outputs = keras.layers.Dense(input_length, activation='sigmoid')(upscale_4_flatten)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [110]:
model = encoder_lstm_decoder(X[0].shape, 8)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy(),
                       tf.keras.metrics.FalseNegatives()])
print(model.summary())
model.fit(x=X, y=X, epochs=1)


Model: "model_21"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_52 (InputLayer)          [(None, 1024, 169,   0           []                               
                                1)]                                                               
                                                                                                  
 conv2d_521 (Conv2D)            (None, 1024, 169, 1  11          ['input_52[0][0]']               
                                )                                                                 
                                                                                                  
 batch_normalization_609 (Batch  (None, 1024, 169, 1  4          ['conv2d_521[0][0]']             
 Normalization)                 )                                                          

InvalidArgumentError: Graph execution error:

Detected at node 'model_21/tf.reshape_77/Reshape' defined at (most recent call last):
    File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/nash/.local/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/nash/.local/lib/python3.8/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/home/nash/.local/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/home/nash/.local/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.8/asyncio/base_events.py", line 570, in run_forever
      self._run_once()
    File "/u